In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install atari_py

In [ ]:
import numpy as np
import random
from collections import deque

import matplotlib.pyplot as plt
from PIL import Image
import imageio

import os
import gym
from tqdm import tqdm

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

In [ ]:
enviroment = gym.make("BreakoutDeterministic-v4").env

In [ ]:
enviroment.reset()

enviroment.step(enviroment.action_space.sample())
pic = enviroment.ale.getScreenRGB()
Image.fromarray(pic)

In [ ]:
Image.fromarray(pic).size

In [ ]:
def resize_gray(img):
    im = Image.fromarray(img)
    im = im.resize((84,84))
    im = im.convert('LA')
    im = np.array(im)
    return im

In [ ]:
resize_gray(pic)

In [ ]:
Image.fromarray(resize_gray(pic))

In [ ]:
Image.fromarray(resize_gray(pic)).size

In [ ]:
class Agent(object):
    def __init__(self, enviroment, optimizer, image_shape):
        
        # Initialize atributes
        self._action_size = enviroment.action_space.n
        self._optimizer = optimizer
        self._image_shape = image_shape
        self.enviroment = enviroment
        
        self.expirience_replay = deque(maxlen=100000)
        
        # Initialize discount and exploration rate
        self.gamma = 0.6
        self.epsilon = 0.1
        
        # Build networks
        self.q_network = self._build_compile_model()
        self.target_network = self._build_compile_model()
        self.alighn_target_model()

    def store(self, state, action, reward, next_state, terminated):
        self.expirience_replay.append((state, action, reward, next_state, terminated))
    
    def _update_epsilon(self):
        self.epsilon -= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
    
    def _build_compile_model(self):
        model = Sequential()
        model.add(Conv2D(32, 8, strides=(4, 4), padding="valid",activation="relu", 
                              input_shape = self._image_shape))
        model.add(Conv2D(64, 4, strides=(2, 2), padding="valid", activation="relu",
                              input_shape = self._image_shape))
        model.add(Conv2D(64, 3, strides=(1, 1), padding="valid",activation="relu",
                              input_shape = self._image_shape))
        model.add(Flatten())
        model.add(Dense(512, activation="relu"))
        model.add(Dense(self._action_size))
        huber = Huber()
        model.compile(loss = huber,
                           optimizer=self._optimizer,
                           metrics=["accuracy"])
        return model

    def alighn_target_model(self):
        self.target_network.set_weights(self.q_network.get_weights())
    
    def act(self, frame):
        if np.random.rand() <= self.epsilon:
            return self.enviroment.action_space.sample()
        
        frame = np.expand_dims(np.asarray(frame).astype(np.float64), axis=0)
        
        q_values = self.q_network.predict(frame)
        return np.argmax(q_values[0])
    

    def retrain(self, batch_size):
        minibatch = random.sample(self.expirience_replay, batch_size)
        
        for state, action, reward, next_state, terminated in minibatch:
            
            state = np.expand_dims(np.asarray(state).astype(np.float64), axis=0)
            next_state = np.expand_dims(np.asarray(next_state).astype(np.float64), axis=0)
            
            target = self.q_network.predict(state)
            
            if terminated:
                target[0][action] = reward
            else:
                t = self.target_network.predict(next_state)
                target[0][action] = reward + self.gamma * np.amax(t)
            
            self.q_network.fit(state, target, epochs=1, verbose=0)

In [ ]:
optimizer = Adam(learning_rate=0.01)

state = enviroment.reset()
state = resize_gray(state)
state.shape

agent = Agent(enviroment, optimizer, state.shape)

batch_size = 32
num_of_episodes = 100
timesteps_per_episode = 100
agent.q_network.summary()

In [ ]:
#Training

for e in tqdm(range(0, num_of_episodes)):
    # Reset the enviroment
    state = enviroment.reset()

    # Initialize variables
    reward = 0
    terminated = False

    for timestep in range(timesteps_per_episode):
        state = resize_gray(state)

        # Run Action
        action = agent.act(state)

        # Take action    
        next_state, reward, terminated, info = enviroment.step(action) 
        next_state = resize_gray(next_state)

        agent.store(state, action, reward, next_state, terminated)

        state = next_state

        if terminated:
            agent.alighn_target_model()
            break

        if len(agent.expirience_replay) > batch_size:
            agent.retrain(batch_size)

In [ ]:
#Reconstructing to getting images

total_epochs, total_penalties = 0, 0
num_of_episodes = 1000
enviroment.reset()
counter = 0
frames = []

for e in range(num_of_episodes):
    state = enviroment.reset()
    state = resize_gray(state)
    
    epochs = 0
    penalties = 0
    reward = 0
    total_reward = 0
    
    terminated = False
    
    for timesteps in tqdm(range(timesteps_per_episode)):
        action = agent.act(state)
        state, reward, terminated, info = enviroment.step(action)
        state = resize_gray(state)
        
        if reward == -10:
            penalties += 1
        
        total_reward += reward
        epochs += 1
        
        frames.append(Image.fromarray(state))
        
    total_penalties += penalties
    total_epochs += epochs

In [ ]:
frames

In [ ]:

fp_out = '/kaggle/working/myagent.gif'
frames[0].save(fp=fp_out, format='GIF', append_images=frames,
         save_all=True, duration=200, loop=0)

In [ ]:
#Check "Data" tab on top-right and expand /kaggle/working/. Download the gif.